In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from textblob import Word
import sklearn
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

c:\users\subburam.subbiah\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\subburam.subbiah\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [152]:
data = pd.read_excel("C:/Users/subburam.subbiah/Desktop/Curent Work/BMW Tagging/Classification/Training_set.xlsx")

In [153]:
data.head()

,Feedback,Type
0,Customer stated that when he got the vehicle t...,Defect
1,Customer stated she used to have the remaining...,DTU
2,The customer stated she was having trouble wit...,Defect
3,Customer asked if she had the power tailgate f...,DTU
4,Customer stated there was a bow on the car and...,Defect


In [154]:
#digits
data['Feedback'] = data['Feedback'].apply(lambda x: ''.join([i for i in str(x) if not i.isdigit()]))
#lowercase
data['Feedback'] = data['Feedback'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
#removing url
data['Feedback'] = data['Feedback'].apply(lambda x: " ".join(x for x in str(x).split() if x.startswith("http") == False))
#removing punctuation
data['Feedback'] = data['Feedback'].str.replace('[^\w\s@]','')
#lematization
#data['Mention Content'] = data['Mention Content'].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))
data['Feedback'] = data['Feedback'].apply(lambda x: ''.join([i for i in str(x) if i[0]!= '@']))
data['Feedback'] = data['Feedback'].apply(lambda x: ''.join([i for i in str(x) if i[0]!= '#']))
data['Feedback'] = data['Feedback'].apply(lambda x: ''.join([i for i in str(x) if i[0]!= '$']))
data['Feedback'] = data['Feedback'].apply(lambda x: ''.join([i for i in str(x) if i[0]!= '_']))

In [155]:
stop = stopwords.words('english')
data['Feedback'] = data['Feedback'].apply(lambda x: " ".join(x for x in str(x).lower().split() if x not in stop))

In [156]:
data['Feedback'] = data['Feedback'].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))

In [173]:
X_train, X_test, Y_train, Y_test = sklearn.cross_validation.train_test_split(data['Feedback'], data['Type'], test_size=0.35, random_state=42)
print (X_train.shape, Y_train.shape)
print (X_test.shape, Y_test.shape)

(7913,) (7913,)
(4262,) (4262,)


In [174]:
X_train.head()

8426     customer stated computer vehicle mind customer...
10464         use car short time suggestion recommendation
6414     writer advised adaptive cruise control work sa...
8184     customer stated car great several bmws custome...
8269     customer stated came vehicle noticed wiper bla...
Name: Feedback, dtype: object

In [175]:
Y_train.head().unique()

array(['Wants', 'Neutral', 'DTU', 'Likes', 'Defect'], dtype=object)

# SVM

In [176]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                     ])

# Desicion Tree

In [128]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', tree.DecisionTreeClassifier()),
                     ])

# RandomForest

In [195]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier(n_estimators=1000)),
                     ])

In [196]:
text_clf.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [197]:
predicted = text_clf.predict(X_test)

In [198]:
print(metrics.classification_report(Y_test, predicted))

             precision    recall  f1-score   support

        DTU       0.77      0.78      0.77       956
     Defect       0.82      0.75      0.79       929
      Likes       0.88      0.94      0.91      1385
    Neutral       0.54      0.63      0.59       249
      Wants       0.72      0.65      0.68       743

avg / total       0.79      0.79      0.79      4262



# Validation

In [182]:
val_data = pd.read_excel("C:/Users/subburam.subbiah/Desktop/Curent Work/BMW Tagging/Classification/Validation_set.xlsx")

In [183]:
len(val_data)
val_data.head()

,Feedback,Type
0,"The car originally has fog lights, but it was ...",Neutral
1,The customer feels that the car is not good-lo...,Wants
2,"The car paint is a bit thin, The customer went...",DTU
3,The car paint is easily scraped off.,DTU
4,The car paint is not good enough.,Wants


In [184]:
#digits
val_data['Feedback'] = val_data['Feedback'].apply(lambda x: ''.join([i for i in str(x) if not i.isdigit()]))
#lowercase
val_data['Feedback'] = val_data['Feedback'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
#removing url
val_data['Feedback'] = val_data['Feedback'].apply(lambda x: " ".join(x for x in str(x).split() if x.startswith("http") == False))
#removing punctuation
val_data['Feedback'] = val_data['Feedback'].str.replace('[^\w\s@]','')
#lematization
#val_data['Mention Content'] = val_data['Mention Content'].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))
val_data['Feedback'] = val_data['Feedback'].apply(lambda x: ''.join([i for i in str(x) if i[0]!= '@']))
val_data['Feedback'] = val_data['Feedback'].apply(lambda x: ''.join([i for i in str(x) if i[0]!= '#']))
val_data['Feedback'] = val_data['Feedback'].apply(lambda x: ''.join([i for i in str(x) if i[0]!= '$']))
val_data['Feedback'] = val_data['Feedback'].apply(lambda x: ''.join([i for i in str(x) if i[0]!= '_']))

In [185]:
stop = stopwords.words('english')
val_data['Feedback'] = val_data['Feedback'].apply(lambda x: " ".join(x for x in str(x).lower().split() if x not in stop))
val_data['Feedback'] = val_data['Feedback'].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))

In [186]:
X_val=val_data['Feedback']
Y_val=val_data['Type']

In [199]:
validated = text_clf.predict(X_val)

In [200]:
print(metrics.classification_report(Y_val, validated))

             precision    recall  f1-score   support

        DTU       0.68      0.15      0.25       915
     Defect       0.33      0.11      0.16       214
      Likes       0.47      0.65      0.55       591
    Neutral       0.37      0.69      0.48       493
      Wants       0.54      0.62      0.58      1285

avg / total       0.53      0.48      0.45      3498

